# <h1 style="text-align:center;font-size:36px">***NOTEBOOK DE BAC A SABLE***</h1>

In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import pandas as pd
import geopandas as gp
import numpy as np
from shapely import wkt
import altair as alt
import csv, re, os, statistics, filecmp, unidecode
from math import sqrt, pi, exp
import Connexion_Transfert as ct
import Import_trafics as it
import Donnees_horaires as dh
import Visualisation as vi
import Outils

import sqlalchemy

In [3]:
with ct.ConnexionBdd('local_otv_boulot') as c : 
    df=pd.read_sql("select * from comptage.na_2010_2019_p where gestionnai='Bordeaux Metropole'", c.sqlAlchemyConn)

In [63]:
df_explode=df[['id_comptag', 'id_cpt']].assign(tmp=df.id_cpt.str.split(',')).explode('tmp')

In [17]:
df_explode

,id_comptag,id_cpt,tmp
0,BdxMet-avenue de branne--0.4989;44.8426,789,789
1,BdxMet-cours du general de gaulle--0.6277;44.7555,"225,227",225
1,BdxMet-cours du general de gaulle--0.6277;44.7555,"225,227",227
2,BdxMet-cours henri brunet--0.5558;44.8698,"1514,1515",1514
2,BdxMet-cours henri brunet--0.5558;44.8698,"1514,1515",1515
...,...,...,...
1181,BdxMet-chemin du pas de la cote--0.5392;44.7822,"1759,1760",1759
1181,BdxMet-chemin du pas de la cote--0.5392;44.7822,"1759,1760",1760
1182,BdxMet-chemin du phare--0.6933;44.8521,1574,1574
1183,BdxMet-rue francois villon--0.514;44.8706,1668,1668


In [19]:
df_excel=pd.read_excel(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\donnees_gestionnaires\BordeauxMetropole\historique\2019\base_comptages_routiers_finale.xlsx')

In [32]:
dfv85=df_explode.merge(df_excel.assign(IDENT=df_excel.IDENT.astype(str)), right_on='IDENT', left_on='tmp', how='inner')

In [38]:
dfv85V2=dfv85[['id_comptag', 'id_cpt', 'tmp', 'IDENT', 'V85_VL', 'V85_PL', 'annee']].copy()#.loc[]
dfv85V2['V85_PL']=dfv85V2.V85_PL.apply(lambda x : np.NaN if x in ('NS', 'NC') else float(x))
dfv85V2['V85_VL']=dfv85V2.V85_VL.apply(lambda x : np.NaN if x in ('NS', 'NC') else float(x))

In [64]:
dfv85V2.groupby(['id_cpt','id_comptag', 'annee']).agg({'V85_VL':np.mean, 'V85_PL':np.mean}).to_csv(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\travail\echanges\BM_V85_ponctuel.csv')

In [69]:
df_indic=pd.concat([dfv85V2.groupby(['id_cpt','id_comptag', 'annee']).agg({'V85_VL':np.mean}).reset_index().rename(
            columns={'V85_VL':'valeur'}).assign(indicateur='v85vl'),
          dfv85V2.groupby(['id_cpt','id_comptag', 'annee']).agg({'V85_PL':np.mean}).reset_index().rename(
            columns={'V85_PL':'valeur'}).assign(indicateur='v85pl')])

In [72]:
with ct.ConnexionBdd('local_otv_boulot') as c : 
    df_indic.loc[~df_indic.valeur.isna()].to_sql('vitesses_bm', c.sqlAlchemyConn)

In [75]:
with ct.ConnexionBdd('local_otv_boulot') as c : 
    dfv85V2.groupby(['id_cpt','id_comptag', 'annee']).agg({'V85_VL':np.mean, 'V85_PL':np.mean}).reset_index().to_sql('vitesses_bm_old_schema', c.sqlAlchemyConn)

In [205]:
fichierNord=r'C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\donnees_gestionnaires\ASF\en_cours\TMJM2019_StJeanLuzSud.xls'
dfFichierNord=pd.read_excel(fichierNord, na_values='*', sheet_name=None)

dfFichierNord['EntreeS2'].drop(['DebutPeriode', 'FinPeriode', 'GARE ENT - Gare Péage *', 'GARE ENT - Libellé Gare Péage', 
                                'GARE ENT - Société *', 'DATE ENT - Annee Mois'], axis=1, inplace=True)
dfFichierNord['EntreeS2'].columns=['tmja', 'tmja_vl', 'tmja_pl', 'id_comptag']
dfFichierNord['EntreeS2']['pc_pl']=round(dfFichierNord['EntreeS2'].tmja_pl/dfFichierNord['EntreeS2'].tmja*100, 2)
dfFichierNord['EntreeS2']['mois']=[k for k in it.dico_mois.keys()]
dfFichierNord['SortieS1'].drop(['DebutPeriode', 'FinPeriode', 'GARE SOR - Gare Péage *', 'GARE SOR - Libellé Gare Péage', 
                                'GARE SOR - Société *', 'DATE SOR - Année Mois **'], axis=1, inplace=True)
dfFichierNord['SortieS1'].columns=['tmja', 'tmja_vl', 'tmja_pl', 'id_comptag']
dfFichierNord['SortieS1']['pc_pl']=round(dfFichierNord['SortieS1'].tmja_pl/dfFichierNord['SortieS1'].tmja*100, 2)
dfFichierNord['SortieS1']['mois']=[k for k in it.dico_mois.keys()]
dfFichierNord['EntreeS1'].drop(['Date', 'Libellé Gare', 'Données', 'Nb_JoursValides'], axis=1, inplace=True)
dfFichierNord['EntreeS1'].columns=['tmja', 'tmja_vl', 'tmja_pl', 'id_comptag']
dfFichierNord['EntreeS1']['pc_pl']=round(dfFichierNord['EntreeS1'].tmja_pl/dfFichierNord['EntreeS1'].tmja*100, 2)
dfFichierNord['EntreeS1']['mois']=[k for k in it.dico_mois.keys()]
dfFichierNord['SortieS2'].drop(['Date', 'Libellé Gare', 'Données', 'Nb_JoursValides'], axis=1, inplace=True)
dfFichierNord['SortieS2'].columns=['tmja', 'tmja_vl', 'tmja_pl', 'id_comptag']
dfFichierNord['SortieS2']['pc_pl']=round(dfFichierNord['SortieS2'].tmja_pl/dfFichierNord['SortieS2'].tmja*100, 2)
dfFichierNord['SortieS2']['mois']=[k for k in it.dico_mois.keys()]

dfBase=pd.concat([v for v in dfFichierNord.values()]).dropna()
dfBase['fichier']=os.path.basename(fichierNord)
dfBase['annee']='2019'
dfBase['src']='tableur Vinci'
dfBase['type_veh']='tv/pl'

In [199]:
cpt.insert_bdd('local_otv_boulot', 'comptage', 'comptage',dfBase.drop_duplicates(['id_comptag', 'src', 'annee', 'type_veh'])[['id_comptag', 'src', 'annee', 'type_veh']])

In [200]:
dfIdCptUniqs=cpt.recupererIdUniqComptage(dfBase.id_comptag.tolist(), '2019', 'local_otv_boulot').rename(columns={'id':'id_comptag_uniq'})

In [201]:
dfIdCptUniqs

,id_comptag_uniq,id_comptag,annee
0,53574,64-A63-Sortie2EntreeSens1-197+660,2019
1,53572,64-A63-Sortie2EntreeSens2-197+660,2019
2,53573,64-A63-Sortie2SortieSens1-197+660,2019
3,53575,64-A63-Sortie2SortieSens2-197+660,2019


In [209]:
IndicAgrege=dfBase.groupby(['id_comptag', 'fichier', 'annee']).agg({'tmja':'mean', 'pc_pl':'mean'}).reset_index()
IndicAgrege['tmja']=round(IndicAgrege.tmja)

In [203]:
pd.melt(IndicAgrege.merge(dfIdCptUniqs, on='id_comptag'), id_var=['id_comptag', 'annee', 'fichier'], value_vars=['tmja', 'pc_pl']

In [210]:
cpt.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege',
               cpt.structureBddOld2NewForm(IndicAgrege, '2019',['id_comptag', 'annee', 'fichier'],['tmja', 'pc_pl'], 'agrege'))

In [215]:
cpt.insert_bdd('local_otv_boulot', 'comptage', 'indic_mensuel',
        pd.melt(dfBase.merge(dfIdCptUniqs, on='id_comptag'), id_vars=['id_comptag_uniq', 'mois', 'fichier'], value_vars=['tmja','pc_pl' ], 
                var_name='indicateur', value_name='valeur'))

In [100]:
data=dfFichierNord.dropna(axis=1, how='all').dropna(axis=0, how='any')
data['annee']='2019'
data['fichier']=os.path.basename(fichierNord)
data=data.drop('Nb données trafic tous véh. valides', axis=1).rename(columns={'TMJ mensuel  tous véhicules':'tmja', 
                                                                        'TMJ mensuel véhicules courts':'tmjm_vl',
                                                                        'TMJ mensuel véhicules longs':'tmjm_pl'})
data['pc_pl']=round(data.tmjm_pl.astype(int)/data.tmja.astype(int)*100,2)
data['tmja']=round(data['tmja'])
dataAgrege=data.loc[data['Aut.']=='A63'].copy()
dataMensuel=data.loc[data['Aut.']!='A63'].copy()
dataMensuel['mois']=dataMensuel['Aut.'].apply(lambda x : [k for k, v in it.dico_mois.items() if any([x.split(' ')[0]==v[y].lower() for y in range(1,len(v))] )][0])

In [52]:
#insertion comptage
cpt=it.Comptage('toto')
cpt.insert_bdd('local_otv_boulot', 'comptage', 'comptage',cpt.creer_comptage(dataAgrege.id_comptag.tolist(),'2019', 'tableur Vinci', 'tv/pl'))

In [55]:
#insertion agrege
cpt.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege',
               cpt.structureBddOld2NewForm(dataAgrege, '2019',['id_comptag', 'annee', 'fichier'],['tmja', 'pc_pl'], 'agrege'))

In [48]:
#insertion mensuel
cpt.insert_bdd('local_otv_boulot', 'comptage', 'indic_mensuel',cpt.structureBddOld2NewForm(cpt.df_attr_mens, 
            cpt.annee,['id_comptag', 'annee', 'fichier', 'donnees_type'],['janv', 'fevr', 'mars', 'avri', 'mai', 'juin', 'juil', 'aout', 'sept',
       'octo', 'nove', 'dece'], 'mensuel'))

In [106]:
dfIdCptUniqs=cpt.recupererIdUniqComptage(data.id_comptag.tolist(), '2019', 'local_otv_boulot').rename(columns={'id':'id_comptag_uniq'})

In [103]:
dataMensBdd=pd.melt(dataMensuel, id_vars=['mois', 'id_comptag', 'fichier'], value_vars=['tmja', 'pc_pl'],var_name='indicateur', value_name='valeur')

In [110]:
cpt.insert_bdd('local_otv_boulot', 'comptage', 'indic_mensuel',dataMensBdd.merge(dfIdCptUniqs, on='id_comptag').drop(['id_comptag','annee'], axis=1))

In [109]:
dataMensBdd.merge(dfIdCptUniqs, on='id_comptag').drop(['id_comptag','annee'], axis=1)

,mois,fichier,indicateur,valeur,id_comptag_uniq
0,aout,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,3255.00,53568
1,avri,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,2101.00,53568
2,dece,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,1531.00,53568
3,fevr,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,1777.00,53568
4,janv,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,1637.00,53568
...,...,...,...,...,...
89,mai,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,4.89,53571
90,mars,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,5.58,53571
91,nove,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,5.21,53571
92,octo,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,4.61,53571


In [104]:
dataMensBdd

,mois,id_comptag,fichier,indicateur,valeur
0,aout,64-A63-Sortie3EntreeSens1-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,3255.00
1,avri,64-A63-Sortie3EntreeSens1-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,2101.00
2,dece,64-A63-Sortie3EntreeSens1-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,1531.00
3,fevr,64-A63-Sortie3EntreeSens1-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,1777.00
4,janv,64-A63-Sortie3EntreeSens1-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,1637.00
...,...,...,...,...,...
89,mai,64-A63-Sortie3SortieSens2-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,4.89
90,mars,64-A63-Sortie3SortieSens2-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,5.58
91,nove,64-A63-Sortie3SortieSens2-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,5.21
92,octo,64-A63-Sortie3SortieSens2-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,4.61
